# Preprocessing of eye voxel data
This notebook cleans the eye voxels through nuisance regression of the confound time series estimated by fMRIprep. Written by C.Baldassano, adjusted by M.Nau.




**Step 1: Import libraries**

In [ ]:
# import libraries
import glob
import nibabel as nib
import pandas as pd
import numpy as np
import pickle 
import os
import h5py
from sklearn import linear_model
from scipy import stats
from natsort import natsorted
import matplotlib.pyplot as plt
import re

**Step 2: Adjust the following paths to accomodate your folder structure**

In [ ]:
# set output directory
out_path = 'where/should/the/files/be/written/'

# set path to your fMRIprep confound regressors
path2conf = 'path/to/fmriprep/'

# set path to your eye-voxel files ('mask_*.p')
path2eyes = 'path/to/functional_files/'
participants = natsorted(os.listdir(path2eyes))
print(participants)

**Step 3: Nuisance regression of confound time series**

In [ ]:
# loop over subjects
for subj_id, sub in enumerate(participants):
    print('Processing subject ', sub)

    # make output directory if it does not exist
    out_subdir = os.path.join(out_path, sub)
    if os.path.exists(out_subdir)==False: 
        os.mkdir(out_subdir)

    # find all file
    all_eyes = natsorted(glob.glob(path2eyes + sub + '/mask_rsub*.p'))
    all_conf = natsorted(glob.glob(path2conf + sub + '/func' + '/*.tsv'))

    for run_id, run in enumerate(all_eyes):
        print('      Loading ', run)

        # load eyeball data from pickle file
        with open(run, 'rb') as f:
            in_data = pickle.load(f)

        # limit analysis to eye voxels
        if run_id == 0:
            mask = np.logical_not(np.all(in_data==0,3))
            print(mask.shape)
        in_data = in_data[mask]
        n_TRs = in_data.shape; n_TRs = n_TRs[1]
        print(np.argwhere(mask == 1).shape)

        # load confound parameters
        conf = np.genfromtxt(all_conf[run_id], names=True)
        reg = np.column_stack((conf['trans_x'],
          conf['trans_x_derivative1'],
          conf['trans_y'],
          conf['trans_y_derivative1'],
          conf['trans_z'],
          conf['trans_z_derivative1'],
          conf['rot_x'],
          conf['rot_x_derivative1'],
          conf['rot_y'],
          conf['rot_y_derivative1'],
          conf['rot_z'],
          conf['rot_z_derivative1'],
          conf['csf'],
          conf['white_matter'],
          conf['framewise_displacement'], ))

        # do the data cleaning
        reg = np.nan_to_num(reg)
        regr = linear_model.LinearRegression()
        regr.fit(reg, in_data.T)
        in_data = in_data - np.dot(regr.coef_, reg.T) - regr.intercept_[:, np.newaxis]

        # reshape data to 4D
        out_data = np.zeros((mask.shape[0],mask.shape[1],mask.shape[2], in_data.shape[1]))
        out_data.shape
        for i in range(0, in_data.shape[1]):
            out_data[mask==True, i] = in_data[:,i]
        out_data = out_data.astype('float32')

        # define output file names
        out_name = run.split('/'); 
        out_name = 'clean_' + out_name[-1]
        out_name = out_name.split('.')

        # save pickle file (if preferred over NIFTI)
        # out_p   = os.path.join(out_subdir,out_name[0] + '.p')
        # print('      writing ', out_p)
        # pickle.dump(out_data, open(out_p, 'wb'))

        # save nifti file
        out_nii = os.path.join(out_subdir,out_name[0] + '.nii')
        print('      writing ', out_nii)
        out_data = nib.Nifti1Image(out_data, affine=np.eye(4))
        nib.save(out_data, out_nii)

Step 4: Crop fMRI data to movie and recall onsets and offsets (specific to the Sherlock Movie Viewing Dataset)

In [ ]:
cropping1 = np.array([[20,7],
                      [20,7],
                      [20,6],
                      [6,13],
                      [20,8],
                      [20,7],
                      [20,6],
                      [20,5],
                      [20,4],
                      [20,6],
                      [20,3],
                      [20,5],
                      [20,5],
                      [20,6],
                      [20,3],
                      [20,5]], dtype=int)

cropping2 = np.array([[20,11],
                      [20,7],
                      [20,7],
                      [6,5],
                      [20,7],
                      [20,7],
                      [20,10],
                      [20,7],
                      [20,8],
                      [20,8],
                      [20,6],
                      [20,7],
                      [20,6],
                      [20,4],
                      [20,4],
                      [20,6]], dtype=int)


nv = 14236
D = np.zeros((1976, 14236, 16))
for i in range(16):
    print('Loading', i)
    m1 = h5py.File('preprocessed/sub-%02d.h5' % (i+1,))['sherlockPart1']['D'][:]
    m2 = h5py.File('preprocessed/sub-%02d.h5' % (i+1,))['sherlockPart2']['D'][:]
    D[:,:,i] = np.concatenate((m1[:,cropping1[i,0]:(-1*cropping1[i,1])],
                               m2[:,cropping2[i,0]:(-1*cropping2[i,1])]), axis=1).T

with h5py.File('preprocessed/movie.h5', 'w') as hf:
    hf.create_dataset('D', data=D)
